In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import re, os, csv, pathlib, ast, os.path
import pandas as pd
from statistics import mean, variance


def replace_with_short_forms(s):
    s=s.replace('english','')
    s=s.replace('_female_','[F]')
    s=s.replace('_male_','[M]')
    return s

def shortened_speaker(s):
    speaker,gender,_=s.split('_')
    return speaker[0].upper()+speaker[1:]+'['+gender[0].upper()+']'

def last_name(pth):
    return pathlib.PurePath(pth).name

def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]

def get_each_run(lne):
    return list(map(float, re.findall(': (.+?) -> ', lne)[0].split(' ')))

def get_selection_counts(s):
    return list(map(replace_with_short_forms, re.findall('Counter\\((.+?)\\)', s)))

def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9]+"run_{}/output/test_infer_log.txt".format(run_number)

def get_test(stats_file_path):
    return stats_file_path[:-9]+"run_1/output/test_out.txt"

def WER_test_file(test_file):
    txt_file = open(test_file, 'r')
    lines = txt_file.readlines()
    matched = ""
    for line in lines:
        if "==========>>>>>>Evaluation Greedy WER: " in line:
            txt_file.close()
            return float(line.rstrip().split(": ")[1])
    txt_file.close()
    print("weiowdnio")
    return ""

def get_eta(func, eta):
    return "-n:"+str(float(eta[4:]))

In [7]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/speaker/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# budget = 100


budget = 100
# target = 50

target = 10
features = '39'
# features = 'TRILL'
csv_name = "report_{}_{}_{}.csv".format(budget, target, features)

cols = ['speaker', 'ground', 'function', 'features', 'duration', 'samples', 
        'WER-r1', 'WER-r2', 'WER-r3', 'WER-mean', 'WER-stdev', 'speakers_run1', 'speakers_run2', 'speakers_run3']
df = pd.DataFrame(columns = cols)

speakers = [f.name for f in os.scandir('./') if f.is_dir() and 'english' in f.name]
cnt = 0

for speaker in speakers:
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(speaker)).is_dir()):
        continue
    pick_from = 'all'
    if not(pathlib.Path('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(speaker, pick_from, budget, target)).is_dir()):
        continue
    for function in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(speaker, pick_from, budget, target)):
        if function == "random":
            stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/stats.txt'.format(speaker, pick_from, budget, 
                                                                                                                target, function)                                                                                  
            if not(os.path.isfile(stats_file_path)):
                continue
            stats_file = open(stats_file_path, 'r')
            lines = stats_file.readlines()
            total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
            sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
            sample_total = mean(total_selections)
            duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
            duration_total = mean(total_durations)
            df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
            df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
            df_selections = get_selection_counts(lines[5])
            try:
                wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                df_wer_mean = round(mean(wers), 2)
                df_wer_stdev = round(variance(wers), 3)**0.5
            except:
#                 continue
                print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
                wers = [0,0,0]
                df_wer_mean = 0
                df_wer_stdev = 999
            print(stats_file_path)
            df = df.append(dict(zip(cols, [speaker, pick_from, function, "NA", df_duration, df_samples]+wers
                                    +[df_wer_mean, df_wer_stdev] + df_selections)), ignore_index=True)
            stats_file.close()
            continue
        for eta in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/'.format(speaker, pick_from, budget, target, function)):
            for similarity in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/'.format(speaker, pick_from, 
                                                                                                    budget, target, function, eta)):
                cnt += 1
                stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/{}/{}/stats.txt'.format(speaker, pick_from, budget, 
                                                                                                            target, function, eta, similarity, 
                                                                                                            features)                                                                                  
                if not(os.path.isfile(stats_file_path)):
                    continue
                stats_file = open(stats_file_path, 'r')
                lines = stats_file.readlines()
                total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
                sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
                sample_total = mean(total_selections)
                duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
                duration_total = mean(total_durations)
                df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
                df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
                df_selections = get_selection_counts(lines[5])
                try:
                    wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                    df_wer_mean = round(mean(wers), 2)
                    df_wer_stdev = round(variance(wers), 3)**0.5
                except:
#                     continue
                    print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
                    wers = [0,0,0]
                    df_wer_mean = 0
                    df_wer_var = 999
                df = df.append(dict(zip(cols, [speaker, pick_from, function+get_eta(function, eta), features, df_duration, df_samples]+wers

                                        +[df_wer_mean, df_wer_stdev] + df_selections)), ignore_index=True)
                stats_file.close()
df = df.sort_values(by=['speaker', 'features', 'ground', 'function'], ascending=True, ignore_index=True)
display(df)

./tamil_male_english/manifests/TSS_output/all/budget_100/target_10/FL2MI/eta_1.0/euclidean/39/stats.txt
total selection : 82 82 82 -> 82.00



ValueError: could not convert string to float: '->'

In [23]:
df = df.sort_values(by=['speaker', 'function', 'features', 'ground'], ascending=True, ignore_index=True)
display(df)

,speaker,ground,function,similarity,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,speakers_run1,speakers_run2,speakers_run3
0,Assamese[F],all,GCMI-n:1.0,euclidean,2947.07/2947.07,513.00/513.00,17.32,16.77,16.96,17.02,0.279,{'assamese[F]': 513},{'assamese[F]': 513},{'assamese[F]': 513}
1,Assamese[F],all,random,NA,487.19/2947.20,98.97/579.33,0.00,0.00,0.00,0.00,31.607,"{'manipuri[F]': 104, 'assamese[F]': 97, 'tamil...","{'manipuri[F]': 131, 'tamil[M]': 102, 'assames...","{'manipuri[F]': 118, 'assamese[F]': 110, 'tami..."


In [24]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# speakered selection: 76 76 76 -> 76.00
# speakered duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

#Evaluation Greedy WER: 16.19

df.to_csv(csv_name, index=False)